# Final Project Sequence 2

This sequence is about detecting and tracking pedestrians, cyclists and cars. In the sequence is an occlusion which the code needs to try and track when the objects goes behind that. This should be obtained based on YOLO object detection and the kalman filter we was teached about in lecture 7 about state estimation. This sequence has ground truth which makes it possible to validate performance of the system

# Step 1
Start by looping through the images to get an understanding of what we are working with and the obstacles in the sequence, to gain an idea of how to solve the execise.

In [13]:
#Imports needed for the application
import cv2
import glob
import os
from ultralytics import YOLO 
import numpy as np
import math
import time
from scipy.optimize import linear_sum_assignment
import pandas as pd
from tqdm import tqdm

# Change this path to the one that works for you
img_folder = "34759_final_project_rect/seq_02/image_02/data"

images = sorted(glob.glob(os.path.join(img_folder, "*.png")))
# Check to see if images are found
if not images:
    raise FileNotFoundError("No images found. Check your folder path.")

# Loop through images in the sequence
for i, img_path in enumerate(images):
    frame = cv2.imread(img_path)
    if frame is None:
        print("Failed to load:", img_path)
        continue

    cv2.imshow("Left Camera Sequence", frame) # The left images are the ground truth
    key = cv2.waitKey(30)
    if key == 27:  # ESC
        break

cv2.destroyAllWindows()

# Step 2 
Look at the labels.txt file to get an idea of what the there is inside of it and what to use

In [4]:
cols = ["frame","track_id","type","truncated","occluded","alpha",
        "left","top","right","bottom",
        "height","width","length",
        "x","y","z","rot_y"]

# Use pandas to read the txt file
labels = pd.read_csv("C:/Users/senik/Documents/2. Semester/Perception for Autonomous Systems/Final project/34759_final_project_rect/seq_02/labels.txt", sep=" ", names=cols)


In [5]:
#Pick an image in the sequence to see the ground truth data based on the labels.txt 
frame_id = 0
frame_dets = labels[labels["frame"] == frame_id]
for _, row in frame_dets.iterrows():
    print(f'Type: {row["type"]}       X: {row["x"]}        Y: {row["y"]}          Z: {row["z"]}')


Type: Car       X: 19.26026        Y: 1.775559          Z: 24.51019
Type: Car       X: 16.289924        Y: 1.602928          Z: 23.743018
Type: Car       X: 13.738582        Y: 1.691811          Z: 24.225602
Type: Car       X: 0.723855        Y: 1.091199          Z: 36.838675
Type: Cyclist       X: -2.85975        Y: 1.524929          Z: 8.816594
Type: Pedestrian       X: 2.383949        Y: 1.44522          Z: 10.654481
Type: Pedestrian       X: 3.220755        Y: 1.423812          Z: 11.111037
Type: Pedestrian       X: -2.918292        Y: 1.420417          Z: 14.233725
Type: Pedestrian       X: -8.353782        Y: 1.274824          Z: 18.866601
Type: Pedestrian       X: -7.643764        Y: 1.248348          Z: 19.522313
Type: Pedestrian       X: -11.048167        Y: 1.115481          Z: 20.605951
Type: Pedestrian       X: -10.299781        Y: 1.083621          Z: 20.859352
Type: Pedestrian       X: -9.571061        Y: 1.089269          Z: 21.128756


# Helper functions for yolo model without kalman filter

In [6]:
# This function does the finding of a cyclist based on a person sitting on the bike so the overlap between the bounding box of a person and the bike
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0]); yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2]); yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(1, (boxA[2] - boxA[0])) * max(1, (boxA[3] - boxA[1]))
    boxBArea = max(1, (boxB[2] - boxB[0])) * max(1, (boxB[3] - boxB[1]))
    return interArea / float(boxAArea + boxBArea - interArea + 1e-6)

# Step 3 
Find out how good the yolo model is based on the grounth truth without the kalman filter for tracktion

In [7]:
# Load YOLO
model = YOLO("yolo11l.pt")
allowed_classes = {"person", "car", "bicycle"}

# Metrics for evaluation of the models performance
TP, FP, FN = 0, 0, 0 # TP = True positive, FP = False positive, FN = False negative

for frame_id, frame_path in enumerate(images):

    # Read frame
    frame = cv2.imread(frame_path)
    if frame is None:
        continue

    frame_for_yolo = frame.copy() # clean input to YOLO
    frame_disp = frame.copy() # used only for drawing


    # Ground truth
    frame_gt = labels[labels["frame"] == frame_id]
    gt_boxes, gt_labels = [], []

    for _, row in frame_gt.iterrows():
        cls = row["type"].lower()
        if cls in {"car", "pedestrian", "cyclist"}:
            box = [row["left"], row["top"], row["right"], row["bottom"]]
            gt_boxes.append(box)
            gt_labels.append(cls)

            # Draw Ground truth bonding boxes
            cv2.rectangle(frame_disp, 
                          (int(box[0]), int(box[1])), 
                          (int(box[2]), int(box[3])), 
                          (0, 255, 0), 2)
            
            # Draw the label above the bounding box
            cv2.putText(frame_disp, f"GT {cls}", 
                        (int(box[0]), int(box[1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 255, 0), 1)


    # Start yolo detection on the current frame
    results = model(frame_for_yolo)

    raw = []
    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy() # Extracts the boundingbox coordinates into a numoy array
        confs = r.boxes.conf.cpu().numpy() # Extracsts the confidence of the detections
        clss = r.boxes.cls.cpu().numpy() # Extracts the class label for the detections

        for box, conf, cls in zip(boxes, confs, clss):
            label = model.names[int(cls)]
            if label in allowed_classes and conf >= 0.5: # Remove objects detected under a confidence of 50%
                raw.append((box, conf, label))


    # Merge cyclists which is a person on a bike
    used = set()
    preds = []

    for i, (box1, conf1, label1) in enumerate(raw):
        if label1 != "person":
            continue

        for j, (box2, conf2, label2) in enumerate(raw):
            if label2 != "bicycle" or j in used:
                continue

            if iou(box1, box2) > 0.2: #Threshold for when it is a cyclist
                x1 = min(int(box1[0]), int(box2[0]))
                y1 = min(int(box1[1]), int(box2[1]))
                x2 = max(int(box1[2]), int(box2[2]))
                y2 = max(int(box1[3]), int(box2[3]))
                # Add cyclists to the detections
                preds.append(([x1, y1, x2, y2], (conf1 + conf2)/2, "cyclist"))
                used.add(i)
                used.add(j)


    # Add the detections of cars and pedestrians
    for k, (box, conf, label) in enumerate(raw):
        if k in used:
            continue

        if label == "car":
            # Add cars to the detections
            preds.append((list(map(int, box)), conf, "car"))
        elif label == "person":
            # Add pedestrians to the detections
            preds.append((list(map(int, box)), conf, "pedestrian"))


    # Draw a bounding box around the detections
    for box, conf, cls in preds:
        x1, y1, x2, y2 = map(int, box)
        color = (0, 255, 255) if cls == "cyclist" else (255, 0, 0)

        cv2.rectangle(frame_disp, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame_disp, f"{cls} {conf:.2f}", 
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    color, 1)


    # Make evaluation of the model
    matched_gt = set()

    for box, conf, cls in preds:
        best_iou = 0
        best_idx = None

        for gi, gt_box in enumerate(gt_boxes):
            i = iou(box, gt_box)
            if i > best_iou:
                best_iou = i
                best_idx = gi

        if best_iou >= 0.5: #Threshold for overlapping between ground truth and yolo detections to find out if the detection is correct
            TP += 1
            matched_gt.add(best_idx)

            # Draw TP marker
            cv2.putText(frame_disp, "TP", 
                        (int(box[0]), int(box[1]) - 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (255, 0, 255), 1)
        else:
            FP += 1

    FN += len(gt_boxes) - len(matched_gt)


    # Show the frames in the sequence as a video
    cv2.imshow("Detections + Ground Truth", frame_disp)
    if cv2.waitKey(30) == 27:
        break


cv2.destroyAllWindows()


# Print evaluations for the model
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0
mAP = TP/(TP+FP+FN)

print("-" * 30)
print(f"TOTAL FRAMES: {frame_id + 1}")
print(f"TP={TP}, FP={FP}, FN={FN}")
print("-" * 30)
print(f"Precision = {precision:.3f}")
print(f"Recall    = {recall:.3f}")
print(f"F1-Score  = {f1_score:.3f}")
print(f"mAP  = {mAP:.3f}")
print("-" * 30)


0: 224x640 8 persons, 2 bicycles, 3 cars, 1 motorcycle, 425.4ms
Speed: 18.6ms preprocess, 425.4ms inference, 16.1ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 8 persons, 2 bicycles, 3 cars, 1 motorcycle, 398.6ms
Speed: 1.5ms preprocess, 398.6ms inference, 3.5ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 7 persons, 2 bicycles, 3 cars, 1 motorcycle, 1 backpack, 390.9ms
Speed: 1.4ms preprocess, 390.9ms inference, 3.3ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 7 persons, 2 bicycles, 4 cars, 1 motorcycle, 400.3ms
Speed: 1.8ms preprocess, 400.3ms inference, 3.2ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 7 persons, 2 bicycles, 3 cars, 1 motorcycle, 349.4ms
Speed: 1.3ms preprocess, 349.4ms inference, 3.2ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 10 persons, 2 bicycles, 5 cars, 1 motorcycle, 337.4ms
Speed: 1.4ms preprocess, 337.4ms inference, 4.7ms postprocess per image at shape (1, 3, 224, 640)

0: 224

# Step 3
Implement the yolo model find the prediction with a kalman filter

# Make helper functions

In [8]:
# Here is the helper functions that is needed for the yolo model with the kalman filter
def iou_xyxy(box1, box2):
    # This function is to see how much two bounding boxes is overlapping
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    # The small value is to make sure the system does not get an error of devision by 0
    return inter_area / (union_area + 1e-9) 

def xyxy_to_cxcywh(box):
    # This function turns the bounding box from corner positions into center position, width and height
    w = box[2] - box[0]
    h = box[3] - box[1]
    cx = box[0] + w/2
    cy = box[1] + h/2
    return np.array([cx, cy, w, h])

def cxcywh_to_xyxy(cx, cy, w, h):
    # This function turns the bounding box from center position, width and height into corner positions 
    x1 = cx - w/2
    y1 = cy - h/2
    x2 = cx + w/2
    y2 = cy + h/2
    return [x1, y1, x2, y2] 

def associate_tracks_dets(tracks, dets_xyxy, dets_cls, iou_thresh=0.3):
    # This functions matches existing object tracks with new detections to see if the new detections matches the existing one
    if len(tracks) == 0:
        return [], [], list(range(len(dets_xyxy)))

    # IoU between tracktions and detections
    iou_matrix = np.zeros((len(tracks), len(dets_xyxy)), dtype=np.float32)
    for t, tr in enumerate(tracks):
        t_box = tr.get_xyxy()
        for d, d_box in enumerate(dets_xyxy):
            if tr.cls != dets_cls[d]:
                iou_matrix[t, d] = 0.0
            else:
                iou_matrix[t, d] = iou_xyxy(t_box, d_box)
                
    matches = []
    unmatched_tracks = set(range(len(tracks)))
    unmatched_dets = set(range(len(dets_xyxy)))

    # Make matching based on IoU
    if iou_matrix.size > 0:
        indices = np.dstack(np.unravel_index(np.argsort(-iou_matrix.ravel()), iou_matrix.shape))[0]
        for t, d in indices:
            if iou_matrix[t, d] < iou_thresh:
                break
            if t in unmatched_tracks and d in unmatched_dets:
                matches.append((t, d))
                unmatched_tracks.remove(t)
                unmatched_dets.remove(d)
                
    return matches, list(unmatched_tracks), list(unmatched_dets)

# Kalman filter function

In [9]:
class KalmanBoxTracker:
    # This class is the kalman filter for tracking the bounding boxes
    def __init__(self, init_z, cls, dt, init_velocity=None):
        #This function is for initializing the kalman filter
        
        self.dt = float(dt)

        #This is the state vector
        self.x = np.zeros((8,1), dtype=np.float32)
        
        self.x[0:4,0] = init_z.reshape(4) # initialize position and size
        if init_velocity is not None:
            self.x[4:8,0] = np.reshape(init_velocity, (4,)) # initialize velocity
        
        self.F = np.eye(8, dtype=np.float32) # State transition matrix
        for i in range(4): 
            self.F[i, i+4] = self.dt

        # Measurement matrix
        self.H = np.zeros((4,8), dtype=np.float32)
        self.H[0,0] = self.H[1,1] = self.H[2,2] = self.H[3,3] = 1.0

        # Uncertainty Covariance matrix
        self.P = np.eye(8, dtype=np.float32) * 10.0

        # Process noise 
        self.Q = np.eye(8, dtype=np.float32)
        self.Q[0,0]=0.5; self.Q[1,1]=0.5 # cx, cy
        self.Q[2,2]=0.1; self.Q[3,3]=0.1 # w, h
        self.Q[4,4]=2.0; self.Q[5,5]=2.0 # vx, vy
        self.Q[6,6]=1.0; self.Q[7,7]=1.0 # vm, vh

        # Measurement noise
        self.R = np.eye(4, dtype=np.float32) * 2.0
        self.R[2,2] = 10.0  # Increased noise for width measurement
        self.R[3,3] = 10.0  # Increased noise for height measurement
        
        # Identity matrix
        self.I = np.eye(8, dtype=np.float32)

        # Initialization of the needed data
        self.cls = cls # object class
        self.id = None # ID of the tracked object
        self.age = 0 # The number of frames since first detection
        self.miss = 0 # The number of frames where it is not seen
        self.score = 0.0 # The confidence of the detection
        self.hits = 0 # The number of times where it has been hit
        self.confirmed = False # Confirmation of the tracktion
        self.last_detection_box = cxcywh_to_xyxy(self.x[0,0], self.x[1,0], self.x[2,0], self.x[3,0])
        self.last_update_frame = -1

    def predict(self):
        # This function predict the next state 
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        self.age += 1

        # Reset if the detection is missed
        if self.miss > 0:
            self.x[6, 0] = 0.0 # vw (velocity width)
            self.x[7, 0] = 0.0 # vh (velocity height)
            
        return self.get_xyxy()

    def update(self, z, score, frame_idx=None):
        # This is the update function of the kalman filter
        z = z.reshape(4,1)
        y = z - (self.H @ self.x)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + (K @ y)
        self.P = (self.I - K @ self.H) @ self.P
        
        self.miss = 0
        self.score = float(score)
        self.hits += 1
        if self.hits >= 2:
            self.confirmed = True
        
        self.last_detection_box = cxcywh_to_xyxy(float(self.x[0,0]), float(self.x[1,0]), 
                                                 float(self.x[2,0]), float(self.x[3,0]))
        self.last_update_frame = frame_idx if frame_idx is not None else self.last_update_frame

    def get_xyxy(self):
        # This function is to get the current bounding boxes
        cx = float(self.x[0,0]); cy = float(self.x[1,0])
        w = max(1.0, float(self.x[2,0])); h = max(1.0, float(self.x[3,0]))
        
        # This is to avoid unrealistic bounding boxes
        max_ratio = 10.0 
        
        if w > h * max_ratio: w = h * max_ratio
        if h > w * max_ratio: h = w * max_ratio
            
        return np.array(cxcywh_to_xyxy(cx, cy, w, h), dtype=np.float32)

In [12]:
# Here is the config for the object detection and tracking using both yolo and kalman filter
MODEL_PATH = "yolo11l.pt"
FPS = 30.0
DT = 1.0 / FPS
IOU_MATCH_THRESH = 0.5      
IOU_MERGE_CYCLIST = 0.2     
CENTER_DIST_GATING = 500.0    # High value allows jumping over large occlusions
INIT_CONF_THRESH = 0.65       # High threshold to prevent tracking noise
SCORE_DECAY = 0.98            # Very slow decay for long memory
PATIENTS = {"car": 80, "cyclist": 100, "pedestrian": 150}
MAX_INIT_PREV_DIST_SQ = 400.0**2
BOX_SIZE_THRESHOLDS = {
    "pedestrian": {"min": 50,  "max": 1000},    
    "car":        {"min": 2000, "max": 25000},   
    "cyclist":    {"min": 100, "max": 15000},   
}

# -------------------- MAIN --------------------
model = YOLO(MODEL_PATH)
allowed_classes = {"person", "car", "bicycle"}
TP, FP, FN = 0, 0, 0
tracks = []
next_id = 1
prev_frame_preds = [] 
tracked_results = [] # List to store output data

# Loop through images
for frame_id, frame_path in enumerate(images):
    frame = cv2.imread(frame_path)
    if frame is None: continue

    frame_disp = frame.copy() # This is for visualization

    # Ground truth
    frame_gt = labels[labels["frame"] == frame_id]
    gt_boxes = []
    for _, row in frame_gt.iterrows():
        cls = row["type"].lower()
        if cls in {"car", "pedestrian", "cyclist"}:
            gt_boxes.append([row["left"], row["top"], row["right"], row["bottom"]])
            # The ground truth drawing (Green boundin box)
            cv2.rectangle(frame_disp, (int(row["left"]), int(row["top"])),
                       (int(row["right"]), int(row["bottom"])), (0,255,0), 2)
            label = f"GT"
            cv2.putText(frame_disp, label, (int(row["left"]), int(row["top"])-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 1)


    # Start yolo detection
    results = model(frame, verbose=False)
    raw = []
    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy() # Extracts the boundingbox coordinates into a numoy array
        confs = r.boxes.conf.cpu().numpy() # Extracsts the confidence of the detections
        clss = r.boxes.cls.cpu().numpy() # Extracts the class label for the detections
        
        for box, conf, cls_idx in zip(boxes, confs, clss):
            label = model.names[int(cls_idx)]
            if label in allowed_classes and float(conf) >= 0.3:
                raw.append((box, float(conf), label))
    
    # Filter by the size
    filtered = []
    for box, conf, label in raw:
        area = (box[2]-box[0]) * (box[3]-box[1])
        if label in BOX_SIZE_THRESHOLDS:
            if area < BOX_SIZE_THRESHOLDS[label]["min"] or area > BOX_SIZE_THRESHOLDS[label]["max"]:
                continue
        filtered.append((box, conf, label))
    raw = filtered

    # Make the cyclists based on a person on top of a bicycle
    used = set()
    preds = []
    for i, (b1, c1, l1) in enumerate(raw):
        if l1 != "person": continue
        for j, (b2, c2, l2) in enumerate(raw):
            if l2 != "bicycle" or j in used: continue
            if iou_xyxy(b1, b2) > IOU_MERGE_CYCLIST:
                # Make it into one big bounding box around the cyclist
                x1 = min(b1[0], b2[0]); y1 = min(b1[1], b2[1])
                x2 = max(b1[2], b2[2]); y2 = max(b1[3], b2[3])
                preds.append(([x1, y1, x2, y2], (c1+c2)/2, "cyclist"))
                used.add(i); used.add(j)
                
    # Add the other detections
    for k, (box, conf, label) in enumerate(raw):
        if k in used: continue
        if label == "car": preds.append((list(map(int, box)), conf, "car"))
        elif label == "person": preds.append((list(map(int, box)), conf, "pedestrian"))

    # Start the predictions with the kalman filter
    for tr in tracks: tr.predict()

    # Match the detections to the tracking
    dets_xyxy = [np.array(b, dtype=np.float32) for b, c, l in preds]
    dets_cls = [l for b, c, l in preds]
    dets_z = [xyxy_to_cxcywh(b) for b, c, l in preds]
    matches, unmatched_tracks, unmatched_dets = associate_tracks_dets(tracks, dets_xyxy, dets_cls, IOU_MATCH_THRESH)

    # Start the update with the kalman filter
    for t, d in matches:
        tracks[t].update(dets_z[d], preds[d][1], frame_id)
        tracks[t].cls = dets_cls[d]

    # Remove the matches that is not correct
    for t in unmatched_tracks:
        tr = tracks[t]
        tr.miss += 1
        tr.score *= SCORE_DECAY

        if tr.miss > PATIENTS.get(tr.cls, 60) or tr.score < 0.05:
            tracks[t] = None

    tracks = [t for t in tracks if t is not None]

    # Start new tracking
    prev_by_cls = {}
    for p in prev_frame_preds: prev_by_cls.setdefault(p["cls"], []).append(p)

    for d in unmatched_dets:
        if preds[d][1] < INIT_CONF_THRESH: continue
        box = dets_xyxy[d]
        # Remove dublicates
        if any(iou_xyxy(t.get_xyxy(), box) > 0.5 for t in tracks): continue
        
        init_vel = None
        cls = dets_cls[d]
        if cls in prev_by_cls:
            cx, cy, _, _ = dets_z[d]
            # Estimate the velocity
            best = min(prev_by_cls[cls], key=lambda p: (cx-p["cx"])**2 + (cy-p["cy"])**2, default=None)
            if best:
                d2 = (cx-best["cx"])**2 + (cy-best["cy"])**2
                if d2 < MAX_INIT_PREV_DIST_SQ:
                    init_vel = [(cx-best["cx"])/DT, (cy-best["cy"])/DT, 0.0, 0.0]
                    
        # Create the new tracktion
        tr = KalmanBoxTracker(dets_z[d], cls, DT, init_vel)
        tr.id = next_id; next_id += 1
        tr.score = preds[d][1]
        tracks.append(tr)

    # Remove the overlapping tracktions
    to_rem = set()
    for i in range(len(tracks)):
        if i in to_rem: continue
        for j in range(i+1, len(tracks)):
            if j in to_rem: continue
            if iou_xyxy(tracks[i].get_xyxy(), tracks[j].get_xyxy()) > 0.4:
                if tracks[i].confirmed != tracks[j].confirmed:
                    to_rem.add(j if tracks[i].confirmed else i)
                else:
                    to_rem.add(j if tracks[i].score > tracks[j].score else i)
    tracks = [t for k, t in enumerate(tracks) if k not in to_rem]
    
    #Start the evaluation and the visualization
    EVAL_SCORE_THRESH = 0.3
    eval_tracks = [t for t in tracks if t.score > EVAL_SCORE_THRESH]
    
    matched_gt_indices = set()
    
    for tr in eval_tracks:
        box = tr.get_xyxy().astype(int)
        pred_box = tr.get_xyxy()
        best_iou = 0
        best_gt_idx = -1
        for idx, gt_box in enumerate(gt_boxes):
            iou = iou_xyxy(pred_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = idx
        
        # Drawing
        is_tp = best_iou >= 0.5
        is_active = tr.miss == 0
        
        color = (0, 0, 255) # Predicted bounding boxes is red
        
        if is_tp:
            TP += 1
            matched_gt_indices.add(best_gt_idx)
            label = f"ID:{tr.id} S:{tr.score:.2f} [Predicted]"
            
        elif not is_tp and is_active:
            FP += 1
            

        # Save data
        tracked_results.append({
            'frame': frame_id,
            'id': tr.id,
            'class': tr.cls,
            'score': tr.score,
            'left': box[0],
            'top': box[1],
            'right': box[2],
            'bottom': box[3],
            'metric_status': 'TP' if is_tp else 'FP'
        })

        # Draw the evaluated track 
        cv2.rectangle(frame_disp, (box[0], box[1]), (box[2], box[3]), color, 2)
        cv2.putText(frame_disp, label, (box[0], box[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    
    FN += len(gt_boxes) - len(matched_gt_indices)

    # Save for next velocity
    prev_frame_preds = []
    for box, conf, cls in preds:
        z = xyxy_to_cxcywh(np.array(box))
        prev_frame_preds.append({"cx": z[0], "cy": z[1], "w": z[2], "h": z[3], "cls": cls})

    cv2.imshow("Tracker", frame_disp)
    if cv2.waitKey(1) == 27: break

cv2.destroyAllWindows()

# Print the evaluation and save the results
results_df = pd.DataFrame(tracked_results)
results_df.to_csv('tracked_results.csv', index=False)
print("Tracked results saved to tracked_results.csv")

# Print Metrics
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0
mAP = TP/(TP+FP+FN)

print("-" * 30)
print(f"TOTAL FRAMES: {frame_id + 1}")
print(f"TP={TP}, FP={FP}, FN={FN}")
print("-" * 30)
print(f"Precision = {precision:.3f}")
print(f"Recall    = {recall:.3f}")
print(f"F1-Score  = {f1_score:.3f}")
print(f"mAP  = {mAP:.3f}")
print("-" * 30)

Tracked results saved to tracked_results.csv
------------------------------
TOTAL FRAMES: 209
TP=1577, FP=314, FN=1561
------------------------------
Precision = 0.834
Recall    = 0.503
F1-Score  = 0.627
mAP  = 0.457
------------------------------


# Stereo vision

Load the camera calibraation, compute the disparity and estimate 3D coordinates

In [16]:
# Load files (Change paths if needed)
left_folder = "34759_final_project_rect/seq_02/image_02/data/"
right_folder = "34759_final_project_rect/seq_02/image_03/data/"
calib_path = "34759_final_project_rect/calib_cam_to_cam.txt"

def load_projection_matrices(calib_file):
    # Load the projection from the calibration file
    P = {}
    with open(calib_file, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("P_rect_"):
                key, values = line.split(":")
                cam = int(key.split("_")[-1])
                P[cam] = np.array(list(map(float, values.split()))).reshape(3,4)
    return P

P = load_projection_matrices(calib_path)
P_left = P[2]   # P_rect_02
P_right = P[3]  # P_rect_03

f = P_left[0,0]               # focale in pixel
B = abs(P_right[0,3] / f)     # baseline = Tx / f
print(f"Focale: {f:.2f} px, Baseline: {B:.3f} m")

np.set_printoptions(precision=15, suppress=False)
print("P_left (02):\n", P_left)
print("P_right (03):\n", P_right)

def compute_disparity_map(left_img, right_img):
    # Compute the disparity map between the left and right image
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
    
    # SGBM parameters
    min_disp = 0
    num_disp = 64 
    block_size = 9
    
    stereo = cv2.StereoSGBM_create(
        minDisparity=min_disp, 
        numDisparities=num_disp, 
        blockSize=block_size, 
        P1=8 * 3 * block_size**2,
        P2=32 * 3 * block_size**2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32
    )
    
    disparity = stereo.compute(left_gray, right_gray).astype(np.float32) / 16.0
    disparity[disparity < 0] = np.nan 
    
    return disparity

def estimate_depth_from_disparity(disparity_map, bbox):
    # Estimate the depth Z
    left, top, right, bottom = [int(x) for x in bbox]
    
    h, w = disparity_map.shape
    top = max(0, top); bottom = min(h, bottom)
    left = max(0, left); right = min(w, right)
    
    disp_region = disparity_map[top:bottom, left:right]
    valid_disparities = disp_region[~np.isnan(disp_region)]
    
    if len(valid_disparities) < 10:
        return -1.0 
        
    median_disparity = np.median(valid_disparities)
    
    if median_disparity <= 1e-6:
        return -1.0 
        
    Z_pred = BASELINE_X_FX / median_disparity
    
    return Z_pred

def project_3d_to_2d(X_c, Y_c, Z_c, FX, FY, CX, CY):
    #Projects a 3D point (X, Y, Z) to 2D image coordinates (u, v)
    if Z_c <= 0: return None, None
        
    u = int((X_c * FX / Z_c) + CX)
    v = int((Y_c * FY / Z_c) + CY)
    
    return u, v


def calculate_pred_3d_bottom_center(track_row, Z_pred):
    # Calculates the 3D center corresponding to the 2D bounding box bottom
    if Z_pred <= 0:
        return None
        
    # 2D bottom center
    u_c = (track_row["left"] + track_row["right"]) / 2 # horizontal center
    v_bottom = track_row["bottom"]                     # vertical bottom edge
    
    X_pred = Z_pred * (u_c - CX) / FX
    
    Y_pred = Z_pred * (v_bottom - CY) / FY
    
    return [X_pred, Y_pred, Z_pred]

Focale: 707.05 px, Baseline: 0.473 m
P_left (02):
 [[ 7.070493e+02  0.000000e+00  6.040814e+02  4.575831e+01]
 [ 0.000000e+00  7.070493e+02  1.805066e+02 -3.454157e-01]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  4.981016e-03]]
P_right (03):
 [[ 7.070493e+02  0.000000e+00  6.040814e+02 -3.341081e+02]
 [ 0.000000e+00  7.070493e+02  1.805066e+02  2.330660e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  3.201153e-03]]


# Visualization for 3D tracking 
This code make a visualization of the 3D tracking with ground truth and the predictions from kalman filter and yolo model

In [18]:
# Path for import of images, labels, the tracktion and the calibration
IMG_FOLDER_LEFT = "34759_final_project_rect/seq_02/image_02/data"
IMG_FOLDER_RIGHT = "34759_final_project_rect/seq_02/image_03/data" 
LABEL_FILE = "34759_final_project_rect/seq_02/labels.txt"
TRACK_FILE_PATH = "tracked_results.csv"
CALIB_FILE_PATH = "34759_final_project_rect/calib_cam_to_cam.txt"

# Set the global parameters
FX = 707.0493
FY = 707.0493
CX = 604.0814
CY = 180.5066
BASELINE_X_FX = 45.75831 

# Visualization Colors
COLOR_PREDICTED_BOX = (255, 0, 0)       # Blue box for 2D track
COLOR_GT_POINT = (0, 255, 0)            # Green point for Ground Truth 3D center (BOTTOM)
COLOR_PRED_3D_POINT = (0, 0, 255)       # Red point for Predicted 3D center (BOTTOM)

# Load data
cols = ["frame","track_id","type","truncated","occluded","alpha",
        "left","top","right","bottom",
        "height","width","length",
        "x","y","z","rot_y"]

try:
    # Load Ground Truth data 
    labels = pd.read_csv(LABEL_FILE, sep=" ", names=cols)
    
    # Load Predicted Tracks data
    df_tracks = pd.read_csv(TRACK_FILE_PATH)

    if 'frame' not in df_tracks.columns:
        alternative_frame_name = 'Frame' 
        if alternative_frame_name in df_tracks.columns:
            df_tracks = df_tracks.rename(columns={alternative_frame_name: 'frame'})
        elif 'frame' not in df_tracks.columns:
             raise KeyError("'frame'") 
            
except KeyError as e:
    print(f"KeyError: Required column {e} is missing. Please check your {TRACK_FILE_PATH} file content.")
    exit()
except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure files exist and the paths are correct.")
    exit()

allowed_gt_classes = ["Car", "Pedestrian", "Cyclist"]
labels_seq = labels[labels["type"].isin(allowed_gt_classes)].copy()
labels_seq.loc[:, 'type'] = labels_seq['type'].str.lower()
labels_seq_grouped = labels_seq.groupby('frame')
tracks_grouped = df_tracks.groupby('frame')

images_left = sorted(glob.glob(os.path.join(IMG_FOLDER_LEFT, "*.png")))
images_right = sorted(glob.glob(os.path.join(IMG_FOLDER_RIGHT, "*.png")))

if not images_left or len(images_left) != len(images_right):
    raise FileNotFoundError("Image files missing or count mismatch.")

print(f"Loaded {len(images_left)} frames for visualization.")

# Visualization

for frame_id, img_path_left in enumerate(tqdm(images_left, desc="Generating Visualization")):
    img_path_right = images_right[frame_id]
    
    frame_left = cv2.imread(img_path_left)
    frame_right = cv2.imread(img_path_right)
    
    if frame_left is None or frame_right is None:
        continue

    frame_disp = frame_left.copy()
    
    # Compute Disparity Map
    disparity_map = compute_disparity_map(frame_left, frame_right)

    # Draw Ground Truth 3D Centers (Red Dot)
    if frame_id in labels_seq_grouped.groups:
        frame_gt = labels_seq_grouped.get_group(frame_id)
        
        for _, row in frame_gt.iterrows():
            X_c, Y_c, Z_c = row["x"], row["y"], row["z"]
            gt_cls = row['type']
            
            # Project GT 3D center (the bottom point) to 2D image coordinates
            u, v = project_3d_to_2d(X_c, Y_c, Z_c, FX, FY, CX, CY)
            
            if u is not None:
                # Draw Red GT dot
                cv2.circle(frame_disp, (u, v), 5, COLOR_GT_POINT, -1)
                
                # Label with X, Y, Z position (in meters)
                gt_label_line1 = f"GT (BOTTOM): {gt_cls}"
                gt_label_line2 = f"X:{X_c:.1f} Y:{Y_c:.1f}"
                gt_label_line3 = f"Z:{Z_c:.1f} m"
                
                # Draw labels 
                cv2.putText(frame_disp, gt_label_line1, (u + 10, v - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLOR_GT_POINT, 1)
                cv2.putText(frame_disp, gt_label_line2, (u + 10, v), cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLOR_GT_POINT, 1)
                cv2.putText(frame_disp, gt_label_line3, (u + 10, v + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, COLOR_GT_POINT, 1)


    # Draw Predicted 2D Boxes and Predicted 3D BOTTOM Centers (Blue Box & Green Dot)
    if frame_id in tracks_grouped.groups:
        frame_tracks = tracks_grouped.get_group(frame_id)
        
        for _, track_row in frame_tracks.iterrows():
            if not all(col in track_row for col in ["left", "top", "right", "bottom", "id", "class"]):
                continue

            bbox = [track_row["left"], track_row["top"], track_row["right"], track_row["bottom"]]
            track_id = int(track_row['id'])
            cls = track_row['class']
            
            # Depth estimation
            Z_pred_val = estimate_depth_from_disparity(disparity_map, bbox)
            
            # Use the function to estimate the 3D center corresponding to the 2D bottom edge
            pred_3d_center = calculate_pred_3d_bottom_center(track_row, Z_pred_val)
            
            # Draw yolo + kalman filter bounding box
            box_int = [int(x) for x in bbox]
            cv2.rectangle(frame_disp, (box_int[0], box_int[1]), (box_int[2], box_int[3]), COLOR_PREDICTED_BOX, 2)
            
            box_label = f"P:{cls} ID:{track_id}"
            cv2.putText(frame_disp, box_label, (box_int[0], box_int[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PREDICTED_BOX, 1)

            # Draw predicted position
            if pred_3d_center is not None:
                X_pred, Y_pred, Z_pred = pred_3d_center
                
                # Project Predicted 3D center (the BOTTOM point) to 2D image coordinates
                u_pred, v_pred = project_3d_to_2d(X_pred, Y_pred, Z_pred, FX, FY, CX, CY)
                
                if u_pred is not None:
                    # Draw Green Predicted dot
                    cv2.circle(frame_disp, (u_pred, v_pred), 5, COLOR_PRED_3D_POINT, -1)
                    
                    # Label with X, Y, Z position (in meters)
                    pred_label_line1 = f"PRED (BOTTOM):"
                    pred_label_line2 = f"X:{X_pred:.1f} Y:{Y_pred:.1f}"
                    pred_label_line3 = f"Z:{Z_pred:.1f} m"
                    
                    # Draw labels 
                    cv2.putText(frame_disp, pred_label_line1, (u_pred + 10, v_pred - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED_3D_POINT, 1)
                    cv2.putText(frame_disp, pred_label_line2, (u_pred + 10, v_pred - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED_3D_POINT, 1)
                    cv2.putText(frame_disp, pred_label_line3, (u_pred + 10, v_pred), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED_3D_POINT, 1)

    # Display the frame
    cv2.imshow("3D Tracking Visualization (Green=Predicted BOTTOM, Red=GT BOTTOM)", frame_disp)

    if frame_id == 0:
        # Show first frame for 30 seconds (30000 ms) for the video
        key = cv2.waitKey(30000)
    else:
        # Show subsequent frames for 50 ms
        key = cv2.waitKey(50)

    if key == 27:  # ESC key to exit
        break

cv2.destroyAllWindows()

Loaded 209 frames for visualization.


Generating Visualization: 100%|██████████████████████████████████████████████████████| 209/209 [01:09<00:00,  3.00it/s]


# Validation of the 3D positions

In [19]:
results = [] # This array will be filled with the ground truth and the predicted results

for frame_id in range(len(images_left)):
    # Ground truth objects in this frame
    frame_gt = labels_seq_grouped.get_group(frame_id) if frame_id in labels_seq_grouped.groups else pd.DataFrame(columns=cols)
    # Predicted tracks in this frame
    frame_tracks = tracks_grouped.get_group(frame_id) if frame_id in tracks_grouped.groups else pd.DataFrame()

    # Iterate over ground truth objects
    for _, gt_row in frame_gt.iterrows():
        gt_id = gt_row["track_id"]
        gt_cls = gt_row["type"]
        gt_x, gt_y, gt_z = gt_row["x"], gt_row["y"], gt_row["z"]

        if not frame_tracks.empty:
            # Loop through predictions in this frame
            for _, track_row in frame_tracks.iterrows():
                pred_id = int(track_row["id"])
                pred_cls = track_row["class"]

                bbox = [track_row["left"], track_row["top"], track_row["right"], track_row["bottom"]]
                Z_pred_val = estimate_depth_from_disparity(disparity_map, bbox)
                pred_3d_center = calculate_pred_3d_bottom_center(track_row, Z_pred_val)

                if pred_3d_center is not None:
                    X_pred, Y_pred, Z_pred = pred_3d_center
                else:
                    X_pred, Y_pred, Z_pred = None, None, None

                results.append({
                    "frame": frame_id,
                    "gt_id": gt_id,
                    "gt_class": gt_cls,
                    "gt_x": gt_x,
                    "gt_y": gt_y,
                    "gt_z": gt_z,
                    "pred_id": pred_id,          
                    "pred_class": pred_cls,
                    "pred_x": X_pred,
                    "pred_y": Y_pred,
                    "pred_z": Z_pred
                })
        else:
            # No predictions at all for this ground truth 
            results.append({
                "frame": frame_id,
                "gt_id": gt_id,
                "gt_class": gt_cls,
                "gt_x": gt_x,
                "gt_y": gt_y,
                "gt_z": gt_z,
                "pred_id": None,
                "pred_class": None,
                "pred_x": None,
                "pred_y": None,
                "pred_z": None
            })

df_validation = pd.DataFrame(results)
print(df_validation.head(20))
df_validation.to_csv("validation_results.csv", index=False)
print("Validation results saved to validation_results.csv")


    frame  gt_id gt_class       gt_x      gt_y       gt_z  pred_id  \
0       0      0      car  19.260260  1.775559  24.510190        1   
1       0      0      car  19.260260  1.775559  24.510190        2   
2       0      0      car  19.260260  1.775559  24.510190        3   
3       0      0      car  19.260260  1.775559  24.510190        4   
4       0      0      car  19.260260  1.775559  24.510190        5   
5       0      0      car  19.260260  1.775559  24.510190        6   
6       0      0      car  19.260260  1.775559  24.510190        7   
7       0      0      car  19.260260  1.775559  24.510190        8   
8       0      1      car  16.289924  1.602928  23.743018        1   
9       0      1      car  16.289924  1.602928  23.743018        2   
10      0      1      car  16.289924  1.602928  23.743018        3   
11      0      1      car  16.289924  1.602928  23.743018        4   
12      0      1      car  16.289924  1.602928  23.743018        5   
13      0      1    

In [20]:
# Normalize the classes
if "gt_class" in df_validation.columns:
    df_validation["gt_class"] = df_validation["gt_class"].str.lower()
if "pred_class" in df_validation.columns:
    df_validation["pred_class"] = df_validation["pred_class"].str.lower()

# Map the classes
CLASS_MAP = {
    "car": "car",
    "cyclist": "cyclist",
    "bicycle": "cyclist",
    "ped": "pedestrian",
    "person": "pedestrian",
    "pedestrian": "pedestrian"
}
# Helper for mapping the classes 
def map_class(c):
    if pd.isna(c):
        return None
    c = str(c).lower().strip()
    return CLASS_MAP.get(c, c)

# Apply mapping to both ground truth and predictions
df_validation["gt_class_norm"] = df_validation["gt_class"].apply(map_class)
df_validation["pred_class_norm"] = df_validation["pred_class"].apply(map_class)

# Helper to check for axis errors
def compute_axis_errors(gt_x, gt_y, gt_z, pred_x, pred_y, pred_z):
    return (gt_x - pred_x), (gt_y - pred_y), (gt_z - pred_z)

grouped = df_validation.groupby(["gt_id", "gt_class_norm"])
summary_rows = []

for (gt_id, gt_cls), group in grouped:
    # Filter to valid predictions and same class
    group_valid = group.dropna(subset=["pred_id", "pred_x", "pred_y", "pred_z"]).copy()
    group_valid = group_valid[group_valid["pred_class_norm"] == gt_cls]
    
    # For each frame, keep only the prediction with smallest Euclidean error
    best_preds = []
    for frame_id, frame_group in group_valid.groupby("frame"):
        candidates = []
        for _, row in frame_group.iterrows():
            gt_x, gt_y, gt_z = float(row["gt_x"]), float(row["gt_y"]), float(row["gt_z"])
            pred_x, pred_y, pred_z = float(row["pred_x"]), float(row["pred_y"]), float(row["pred_z"])
            err_x, err_y, err_z = compute_axis_errors(gt_x, gt_y, gt_z, pred_x, pred_y, pred_z)
            euclid = np.sqrt(err_x**2 + err_y**2 + err_z**2)
            candidates.append((euclid, row, err_x, err_y, err_z))
        if candidates:
            best_preds.append(min(candidates, key=lambda x: x[0]))

    num_frames_matched = len(best_preds)

    if num_frames_matched == 0:
        # No valid predictions for this ground truth across all frames
        any_row = group.iloc[0]
        print(f"\nGT {gt_cls} (ID={gt_id}) at ({float(any_row['gt_x']):.2f}, {float(any_row['gt_y']):.2f}, {float(any_row['gt_z']):.2f}) over all frames has 0 predictions:")
        summary_rows.append({
            "gt_id": gt_id,
            "gt_class": gt_cls,
            "frames_matched": 0,
            "mae_x": np.nan,
            "mae_y": np.nan,
            "mae_z": np.nan,
            "mean_euclid": np.nan
        })
        continue

    # Collect signed errors per frame
    errors_x, errors_y, errors_z, euclids = [], [], [], []

    header_gt_x = float(best_preds[0][1]["gt_x"])
    header_gt_y = float(best_preds[0][1]["gt_y"])
    header_gt_z = float(best_preds[0][1]["gt_z"])
    print(f"\nGT {gt_cls} (ID={gt_id}) at ({header_gt_x:.2f}, {header_gt_y:.2f}, {header_gt_z:.2f}) over all frames has {num_frames_matched} matched predictions:")

    for euclid, row, err_x, err_y, err_z in best_preds:
        pred_id = int(row["pred_id"])
        pred_x, pred_y, pred_z = float(row["pred_x"]), float(row["pred_y"]), float(row["pred_z"])

        # Print signed errors per axis for this frame
        print(f"  - Frame {int(row['frame'])}, Pred ID={pred_id}, class={row['pred_class_norm']}, "
              f"pos=({pred_x:.2f}, {pred_y:.2f}, {pred_z:.2f}), "
              f"error_x={err_x:.2f}, error_y={err_y:.2f}, error_z={err_z:.2f}, euclid={euclid:.2f}")

        errors_x.append(err_x); errors_y.append(err_y); errors_z.append(err_z); euclids.append(euclid)

    # Report MAE (mean absolute error) per axis and mean euclidean error across frames
    mae_x = float(np.mean(np.abs(errors_x)))
    mae_y = float(np.mean(np.abs(errors_y)))
    mae_z = float(np.mean(np.abs(errors_z)))
    mean_euclid = float(np.mean(euclids))

    print(f"  => MAE for GT ID={gt_id}: x={mae_x:.2f}, y={mae_y:.2f}, z={mae_z:.2f} | mean_euclid={mean_euclid:.2f}")

    summary_rows.append({
        "gt_id": gt_id,
        "gt_class": gt_cls,
        "frames_matched": num_frames_matched,
        "mae_x": mae_x,
        "mae_y": mae_y,
        "mae_z": mae_z,
        "mean_euclid": mean_euclid
    })

# Save the summery table
df_summary = pd.DataFrame(summary_rows).sort_values(["gt_class", "gt_id"])
df_summary.to_csv("gt_error_summary.csv", index=False)
print("\n Saved per-GT error summary to gt_error_summary.csv")

# Predictions with no ground truth or class mismatch
gt_keys = set(df_validation[["frame", "gt_id", "gt_class_norm"]].dropna().apply(tuple, axis=1))
pred_rows = df_validation.dropna(subset=["pred_id", "pred_x", "pred_y", "pred_z"])

preds_no_gt = []
preds_class_mismatch = []

for _, row in pred_rows.iterrows():
    key = (row["frame"], row["gt_id"], row["gt_class_norm"])
    if pd.isna(row["gt_id"]) or key not in gt_keys:
        preds_no_gt.append(row)
    else:
        if row["pred_class_norm"] != row["gt_class_norm"]:
            preds_class_mismatch.append(row)

if len(preds_no_gt) == 0:
    print("\nNo predictions without GT correspondence.")
else:
    print("\nPredictions with no GT correspondence:")
    for _, r in pd.DataFrame(preds_no_gt).iterrows():
        print(f"  - Frame {int(r['frame'])}, Pred ID={int(r['pred_id'])}, class={r['pred_class_norm']}, "
              f"pos=({float(r['pred_x']):.2f}, {float(r['pred_y']):.2f}, {float(r['pred_z']):.2f})")

if len(preds_class_mismatch) == 0:
    print("\nNo predictions with class mismatch against GT.")
else:
    print("\nPredictions with class mismatch against GT:")
    for _, r in pd.DataFrame(preds_class_mismatch).iterrows():
        print(f"  - Frame {int(r['frame'])}, GT ID={int(r['gt_id'])}, GT class={r['gt_class_norm']} "
              f"=> Pred ID={int(r['pred_id'])}, Pred class={r['pred_class_norm']}, "
              f"pos=({float(r['pred_x']):.2f}, {float(r['pred_y']):.2f}, {float(r['pred_z']):.2f})")



GT car (ID=0) at (19.26, 1.78, 24.51) over all frames has 209 matched predictions:
  - Frame 0, Pred ID=6, class=car, pos=(2.39, 0.18, 3.01), error_x=16.87, error_y=1.59, error_z=21.50, euclid=27.37
  - Frame 1, Pred ID=6, class=car, pos=(2.39, 0.18, 3.01), error_x=16.87, error_y=1.59, error_z=21.50, euclid=27.37
  - Frame 2, Pred ID=6, class=car, pos=(2.39, 0.18, 3.01), error_x=16.87, error_y=1.59, error_z=21.50, euclid=27.37
  - Frame 3, Pred ID=6, class=car, pos=(2.39, 0.18, 3.01), error_x=16.87, error_y=1.59, error_z=21.50, euclid=27.37
  - Frame 4, Pred ID=6, class=car, pos=(2.38, 0.18, 3.00), error_x=16.88, error_y=1.59, error_z=21.51, euclid=27.39
  - Frame 5, Pred ID=6, class=car, pos=(2.38, 0.18, 3.00), error_x=16.88, error_y=1.59, error_z=21.51, euclid=27.39
  - Frame 6, Pred ID=6, class=car, pos=(2.38, 0.18, 3.00), error_x=16.88, error_y=1.59, error_z=21.51, euclid=27.39
  - Frame 7, Pred ID=6, class=car, pos=(2.39, 0.18, 3.01), error_x=16.87, error_y=1.59, error_z=21.50, e